In [14]:
from pandas import DataFrame, read_csv
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split

X_file = r'dataset/X.csv'
X = pd.read_csv(X_file)

Y_file = r'dataset/Y.csv'
Y = pd.read_csv(Y_file)

pd.set_option('display.max_columns', len(X.columns))

In [15]:
X.shape, Y.shape

((3874, 25), (3874, 1))

In [16]:
from sklearn_pandas import DataFrameMapper
from sklearn.feature_extraction.text import TfidfVectorizer

mapper = DataFrameMapper([
    ('adult', None),
    ('belongs_to_collection', TfidfVectorizer()), 
    ('budget', None),
    ('genres', TfidfVectorizer()),
    ('homepage', None),
    ('overview', TfidfVectorizer()),
    ('popularity', None),
    ('production_companies', TfidfVectorizer()),
    ('production_countries', TfidfVectorizer()),
#    ('release_date', None),
    ('revenue', None), 
    ('runtime', None),
    ('spoken_languages', None),
    ('tagline', TfidfVectorizer()), 
    ('title', TfidfVectorizer()),
    ('vote_average', None),
    ('vote_count', None),
    ('cast', TfidfVectorizer()),
    ('keywords', TfidfVectorizer()),
    ('cast_size', None),
    ('crew_size', None),
    ('director', TfidfVectorizer()),
    ('producers', TfidfVectorizer()),
    ('executive_producers', TfidfVectorizer()),
])

X.fillna('', inplace=True) # can't have nan in any of the columns

features = mapper.fit_transform(X)

In [17]:
X_tr, X_ts, Y_tr, Y_ts = train_test_split(features, Y, train_size = 0.7)

In [18]:
X_tr.shape, X_ts.shape, Y_tr.shape, Y_ts.shape

((2711, 51092), (1163, 51092), (2711, 1), (1163, 1))

In [19]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

gbt = GradientBoostingClassifier(max_features="log2", learning_rate=0.01, n_estimators=100)
gbt.fit(X_tr, Y_tr)

p = gbt.predict(X_ts)

precision = accuracy_score(p, Y_ts) * 100
print("Accuracy using GB: {0:.2f}%".format(precision))

/home/justin/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


NameError: name 'accuracy_score' is not defined